In [2]:
import json

In [3]:
raw_data = sc.textFile('hdfs://master:54310/clean'). \
    map(lambda line: json.loads(line))

In [5]:
excluding_wrods = raw_data.flatMap(lambda doc: [(f,1) for f in doc['Text']]). \
    reduceByKey(lambda a,b: a+b). \
    filter(lambda wc: wc[1] < 6). \
    collectAsMap()

In [6]:
len(excluding_wrods)

416090

In [7]:
hub_bl = {'лог компании', 'рная дыра', 'пиарюсь'}
def remove_bl_hubs_from_hubs(doc: dict):
    doc['Hubs'] = [h for h in doc['Hubs'] if all([h_bl not in h for h_bl in hub_bl])]
    return doc

def exclude_words(words: list):
    return [w for w in words if w not in excluding_wrods]

    
sc.textFile('hdfs://master:54310/clean'). \
    map(lambda line: remove_bl_hubs_from_hubs(json.loads(line))). \
    filter(lambda doc: doc['Hubs']). \
    map(lambda doc: {'Id': doc['Id'],
                     'Features': exclude_words(doc['Text']),
                     'Label': doc['Hubs']}). \
    map(lambda x: json.dumps(x)). \
    repartition(4). \
    saveAsTextFile('hdfs://master:54310/multi-label')